In [9]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

import numpy as np
import re

In [10]:
batch_size = 64
latent_dim = 256

input_data_path = 'data/aion_train_v.txt'
target_data_path = "data/aion_train_t.txt"

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(input_data_path, "r", encoding="utf-8") as f:
    reader = f.readlines()
    for i in reader:
        input_texts.append(i)
        for char in i:
            if char not in input_characters:
                input_characters.add(char)
            
with open(target_data_path, "r", encoding="utf-8") as f:
    reader = f.readlines()
    for i in reader:
        # '\t'  = start sequence
        # '\n' = end sequence
        target_text = '\t' + i
        target_texts.append(target_text)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

In [11]:
input_characters = sorted(list(input_characters)) ## character를 순서대로 정리 ㄱ~ㅎ
target_characters = sorted(list(target_characters))  ## character를 순서대로 정리 a~z ㄱ~ㅎ
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])  ## input texts에 있는 문장 길이를 순서대로 다른 리스트에 등록하고 최대 찾기
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [13]:
## one hot vector 생성 - [0,1,...0,...0,0]
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

## seq2seq Model load

In [14]:
model = load_model('model_save/aion_slu_lstm.h5')

In [15]:
# plot_model(model, to_file='model.png', show_shapes=True)

In [16]:
encoder_inputs = model.input[0] ##[문장길이, one-hot vector길이]
encoder_outputs, state_h, state_c = model.layers[2].output
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.

decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                     [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [17]:
# plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
# plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == "\n" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Test / 후 처리

In [19]:
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")

def basic_tokenizer(sentence):
  """Very basic tokenizer: split the sentence into a list of tokens."""
  words = []
  #for space_separated_fragment in sentence.strip().split():
  words.extend(_WORD_SPLIT.split(sentence))
  return [w for w in words if w]

In [22]:
test_input_texts = []
char_input_tests= set()

with open("data/aion_test_v.txt", "r", encoding='utf=8') as f:
    reader =f.readlines()
    for i in reader:
        test_input_texts.append(i)
        
test_encoder_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for i in range(10):
    char_input_tests= set()
    
    s = input('문자열을 입력하세요: ')
    s = s.strip()
    for char in s:
        if char not in char_input_tests:
            char_input_tests.add(char)
            
    for t, char in enumerate(s):
        if char in input_characters:
            test_encoder_input_data[i, t, input_token_index[char]] = 1.
        else:
            char= " "
            test_encoder_input_data[i, t, input_token_index[char]] = 1.
            
    char_input_tests.add(" ")
    input_seq = test_encoder_input_data[i: i+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-----------------------------------')
    print('Decoded sentence:', decoded_sentence)
    revised_decoded_sentence = basic_tokenizer(decoded_sentence)
    for t, char in enumerate(revised_decoded_sentence[2]):
        if char not in char_input_tests:
            revised_decoded_sentence[2]=""
    revised_decoded_sentence = "".join(revised_decoded_sentence)
    print('revised Decoded sentence:', revised_decoded_sentence)


문자열을 입력하세요: 야영지 세네모네아의 이동 어서 거점 시작 하겠니
-----------------------------------
Decoded sentence: teleport(세네모네아의 야영지).

revised Decoded sentence: teleport(세네모네아의 야영지).

문자열을 입력하세요: 야영지 이동 세네모네아의 어서
-----------------------------------
Decoded sentence: teleport(세네모네아의 야영지).

revised Decoded sentence: teleport(세네모네아의 야영지).

문자열을 입력하세요: 야영지 세세모세아의 거점 이동 어서하겠니
-----------------------------------
Decoded sentence: teleport(세네모네아의 야영지).

revised Decoded sentence: teleport().



KeyboardInterrupt: 